In [1]:
import sys
sys.path.append("..")
import msgspec
import faker
from tqdm.auto import tqdm
from typing import Any, cast
import pickle
import gc
import random
import datetime

In [2]:
fake = faker.Faker("ru_RU")

In [3]:
from dkvs.bptree import BPTree
BPTree.HALF_INODE_SIZE = 32
BPTree.MAX_INODE_SIZE = BPTree.HALF_INODE_SIZE * 2
BPTree.HALF_LEAF_SIZE = 128
BPTree.MAX_LEAF_SIZE = BPTree.HALF_LEAF_SIZE * 2

In [4]:
class MyKey(msgspec.Struct, frozen=True, order=True):
    full_name: str

class MyValue(msgspec.Struct):
    data: Any

In [5]:
index = BPTree[MyKey, MyValue]()

In [6]:
faker.Faker.seed(1)
fake = faker.Faker("ru_RU")

In [7]:
data = []
gc.collect()

340

In [8]:
%%time
for i in tqdm(range(10_000)):
    random_int = fake.random_int(0, 99_999_999)
    val = f"{random_int:010} " + fake.name()
    result = index.insert(MyKey(val), fake.address())
    if result[0] is None:
        print(f"Duplicate key: {val}: {result[1]}")
    data.append(val)

  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 891 ms, sys: 8.77 ms, total: 900 ms
Wall time: 895 ms


In [9]:
index.validate(), gc.collect()

({}, 18)

In [10]:
index.find(MyKey('xxx'))

(None, None, None)

In [11]:
%%timeit
index.find(MyKey('0001677529 Шаров Леонтий Филатович'))

3.21 µs ± 56 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [12]:
index.values

['ст. Североморск, бул. Воровского, д. 81 к. 166, 439150',
 'клх Арсеньев, алл. Бабушкина, д. 9/4 стр. 4, 740681',
 'д. Калачинск, наб. Тюленина, д. 1/8 к. 76, 585178',
 'с. Балашиха, бул. Льва Толстого, д. 92 к. 40, 685957',
 'с. Белый Яр (Томск.), ш. Радищева, д. 78 к. 6/7, 088995',
 'ст. Усть-Илимск, ул. Кирпичная, д. 107 к. 4, 192541',
 'клх Москва, МГУ, пр. Торговый, д. 646 к. 4/5, 483960',
 'д. Кетченеры, ш. Курчатова, д. 18 стр. 6/7, 423041',
 'г. Ухта, пер. Тамбовский, д. 3/9, 635139',
 'к. Темрюк, ул. 40 лет Победы, д. 5/1 к. 63, 563416',
 'клх Бавлы, алл. Краснодарская, д. 48, 598455',
 'к. Аша, ул. Максима Горького, д. 62 к. 799, 983914',
 'к. Всеволожск, ш. Калинина, д. 290, 103396',
 'с. Уренгой, пер. Прудный, д. 6/2 к. 15, 049576',
 'к. Кашира, ш. Шаумяна, д. 5 стр. 48, 335141',
 'г. Мценск, наб. Лермонтова, д. 25, 330361',
 'ст. Новороссийск, пр. Сенной, д. 285 к. 32, 541894',
 'ст. Юрюзань, пер. Высокий, д. 74 стр. 40, 176848',
 'к. Ступино, ш. Баумана, д. 305, 244696',

In [13]:
index.print_node(index.tree[index.root_node])


node-idx:1, prev:None, next:None, level: 1  <============ *ROOT*
   |-> MyKey(full_name='0001778656 Максимов Гаврила Измаилович'): 0, 48
   |-> MyKey(full_name='0003358040 Пахомов Эрнест Давидович'): 48, 25
   |-> MyKey(full_name='0004631176 Марина Андреевна Шарапова'): 25, 50
   |-> MyKey(full_name='0006047413 Лыткина Милица Оскаровна'): 50, 13
   |-> MyKey(full_name='0007915242 Савва Архипович Панов'): 13, 37
   |-> MyKey(full_name='0009772586 Екатерина Вячеславовна Панфилова'): 37, 20
   |-> MyKey(full_name='0011622887 Варвара Валентиновна Зимина'): 20, 41
   |-> MyKey(full_name='0013332303 Гаврилов Прокл Феликсович'): 41, 8
   |-> MyKey(full_name='0014595629 Зинаида Алексеевна Крылова'): 8, 59
   |-> MyKey(full_name='0016137119 Кондратьев Кузьма Ильич'): 59, 27
   |-> MyKey(full_name='0017703271 Брагина Василиса Никифоровна'): 27, 49
   |-> MyKey(full_name='0019146274 Демид Ермилович Ефимов'): 49, 14
   |-> MyKey(full_name='0020487853 Горшкова Надежда Богдановна'): 14, 51
   |-> M

In [14]:
index.min()[1]

MyKey(full_name='0000017833 Зайцева Валерия Ефимовна')

In [15]:
index.max()[1]

MyKey(full_name='0099994480 Евфросиния Рудольфовна Кузнецова')

In [16]:
data[0:10]

['0018034063 Брагина Ангелина Ждановна',
 '0002996023 Потапов Авксентий Тарасович',
 '0024951916 Владимирова Кира Васильевна',
 '0014486288 Регина Никифоровна Фомина',
 '0036142079 Кошелева Агата Натановна',
 '0061491422 Брагина Елизавета Вячеславовна',
 '0022477484 Лобанов Давыд Егорович',
 '0030249188 Куликов Анатолий Владиславович',
 '0010260429 Лаврентий Владиленович Аксенов',
 '0090483285 Феврония Георгиевна Горшкова']

In [17]:
random.shuffle(data)

In [34]:
%%time
print(len(data))
for i, k in enumerate(data):
    val, _, ptr = index.find(MyKey(k))
    if ptr is not None:
        print(f"{i:09}, found: {ptr} -> {k}, {val}")
    else:
        ptr = index.find(k)
        print(f"Key: {k} not found")

print("Done!")

10000
000000000, found: 2908 -> 0033834733 Лукия Ивановна Жданова, клх Юрьевец (Иван.), алл. Жуковского, д. 9/1 к. 570, 889061
000000001, found: 9349 -> 0002766934 Лазарев Аполлинарий Иларионович, с. Снежногорск (Мурм.), пр. Крупской, д. 6/6, 631339
000000002, found: 6780 -> 0008548653 Аггей Евсеевич Шилов, клх Ногинск (Моск.), ул. Городская, д. 4, 244307
000000003, found: 5895 -> 0068384565 Дорофей Чеславович Наумов, п. Новосибирск, бул. Кооперативный, д. 16 к. 1, 812381
000000004, found: 5977 -> 0005075473 Смирнов Самуил Эдуардович, ст. Новый Оскол, пр. Полтавский, д. 939, 796163
000000005, found: 8507 -> 0010289806 Исаева Майя Васильевна, клх Печора, ш. Водопроводное, д. 4 к. 3, 684563
000000006, found: 4996 -> 0050953230 Медведев Остап Гавриилович, с. Яшалта, наб. Азовская, д. 704, 256113
000000007, found: 1177 -> 0045607917 Сергеев Филарет Игоревич, д. Мокшан, пр. Лермонтова, д. 1 к. 6/4, 269556
000000008, found: 6841 -> 0065224212 Селиверстова Татьяна Рудольфовна, п. Вязьма, наб.